# Step 1: Assemble Behavior
Using the TRANSFORM-DBS MSIT data collected from healthy controls. Here we are just checking over participants to exclude individuals with many missing trials or error responses.

In [ ]:
import os
import numpy as np
from pandas import read_csv
root_dir = '/space/sophia/2/users/DARPA-Behavior/msit/csv'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

observed = 0.90    # 90% or more of possible data must be present
accuracy = 0.90    # 90% accuracy or higher

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

csv_files = sorted([f for f in os.listdir(root_dir) if f.startswith('hc') and f.endswith('mri-1')])
sessid = open('scripts/fsfast/sessid', 'w')

for f in csv_files:
    
    ## Get subject name.
    subject = f.split('_')[0]
    
    ## Load CSV.
    df = read_csv(os.path.join(root_dir,f))
    df = df[df.Condition != 0].reset_index(drop=True)   # Drop rest trials.
    
    ## Assess quality.
    if (df.ResponseAccuracy != 99).mean() < observed: continue
    else: df = df[df.ResponseAccuracy != 99].reset_index(drop=True)
    
    if df.ResponseAccuracy.mean() < accuracy: continue
    else: sessid.write('%s\n' %subject)
    
## Save.
sessid.close()

# Step 2: Construct Regressors
Boxcars made following Grinband et al. (2008).

In [ ]:
import os
import numpy as np
import pylab as plt
from pandas import read_csv
from scipy.special import gammaln
root_dir = '/space/sophia/2/users/DARPA-Behavior/msit/csv'
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
conditions = ['Neu','Int']
n_conditions = len(conditions)

## Timing information.
n_acq = 228
tr = 1.75
sfreq = 1e2

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define useful functions.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def spm_hrf(RT, P=None, fMRI_T=16):
    p = np.array([6, 16, 1, 1, 6, 0, 32], dtype=float)
    if P is not None:
        p[0:len(P)] = P

    _spm_Gpdf = lambda x, h, l: np.exp(h * np.log(l) + (h - 1) * np.log(x) - (l * x) - gammaln(h))
    # modelled hemodynamic response function - {mixture of Gammas}
    dt = RT / float(fMRI_T)
    u = np.arange(0, int(p[6] / dt + 1)) - p[5] / dt
    with np.errstate(divide='ignore'):  # Known division-by-zero
        hrf = _spm_Gpdf(u, p[0] / p[2], dt / p[2]) - _spm_Gpdf(u, p[1] / p[3],
                                                               dt / p[3]) / p[4]
    idx = np.arange(0, int((p[6] / RT) + 1)) * fMRI_T
    hrf = hrf[idx]
    hrf = hrf / np.sum(hrf)
    return hrf

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 

subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for subject in subjects:
    
    print subject,

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Initialize regressors.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    
    ## Setup timing information.
    total_time = n_acq * tr
    times = np.arange(0, total_time+1./sfreq, 1./sfreq)
    n_times = times.shape[0]

    ## Initialize boxcars.
    neural_signal = np.zeros((n_conditions,n_times))
    
    ## Load behavior information.
    df = read_csv(os.path.join(root_dir, '%s_msit_mri-1' %subject))
    df = df[df.Condition != 0]            # Drop rest trials.
    df = df[df.ResponseAccuracy != 99]    # Drop missing trials.
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Generate boxcars.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
    
    for n in range(n_conditions):
        
        ## Extract onsets/offsets.
        onsets  = df.loc[df.Condition==n+1, 'StimOnset'].as_matrix()
        offsets = df.loc[df.Condition==n+1, 'ResponseOnset'].as_matrix()
        
        ## Round onsets/offsets.
        onsets  = onsets.round(np.log10(sfreq).astype(int))
        offsets = offsets.round(np.log10(sfreq).astype(int))
        
        ## Iteratively generate boxcars.
        for onset, offset in zip(onsets,offsets): 
            mask = (times >= onset) & (times <= offset)
            neural_signal[n,mask] = 1

    ## Perform convolution.
    hrf = spm_hrf(1./sfreq)
    bold_signal = np.apply_along_axis(np.convolve, 1, neural_signal, v=hrf)
    bold_signal = bold_signal[:,:neural_signal.shape[-1]] # Set back to original length.
    
    ## Downsample to start of TR.
    tr_onsets = np.insert( np.cumsum( np.ones(n_acq-1)*tr ), 0, 0 )
    ds = np.in1d(times, tr_onsets)
    if not ds.sum() == n_acq: raise ValueError('Oh noes!')
    bold_signal = bold_signal[:,ds]
    
    ## Normalize regressors. [See Calhoun et al. (2004)]
    ## First we normalize [Intercept, DDB, Valence] such that the sum
    ## of their timeseries squared is equal to 1.
    sums = np.power(bold_signal,2).sum(axis=1)
    bold_signal = (bold_signal.T / np.sqrt(sums)).T
    
    ## Save task regressors.
    for arr, label in zip(bold_signal, conditions):

        f = '%s/%s/msit_001/001/afMSIT.%s.par' %(mri_dir,subject,label)
        try: np.savetxt(f, arr[:,np.newaxis], fmt='%s')
        except IOError: pass
    
print 'Done.'

# Step 3: Construct Timepoint Censors

In [ ]:
import os
import numpy as np
from pandas import read_csv
from scipy.signal import detrend
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Timing information.
n_acq = 228
tr = 1.75

## Scrubbing parameters.
thresholds = [0.0, 0.5, 1.0]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define TR onsets.
tr_onsets = np.insert( np.cumsum( np.ones(n_acq - 1) * tr ), 0, 0 )

## Get subjects list.
subjects = np.loadtxt('scripts/fsfast/sessid', dtype='str')
info = open('fmri/nuisance_info.csv','w')
info.write('Subject,n_mc,FD=0.0,FD=0.5,FD=1.0\n')

for subject in subjects:
    
    info.write('%s,' %subject)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute framewise displacement.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Read motion data.
    mc = os.path.join(mri_dir, subject, 'msit_001', '001', 'fmcpr.mcdat')
    try: mc = np.loadtxt(mc)[:,1:7]
    except IOError: 
        print 'Drop %s.' %subject
        continue

    ## Invert angular displacement.
    fd = mc.copy()
    fd[:,:3] = np.deg2rad(fd[:,:3]) 
    fd[:,:3] *= 50

    ## Compute framewise displacement (See Power 2012, 2014).
    fd = np.insert( np.abs( np.diff(fd, axis=0) ).sum(axis=1), 0, 0 )

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Compute motion regressors.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Remove trends.
    mc = detrend(mc, axis=0, type='constant')
    mc = detrend(mc, axis=0, type='linear')
    
    ## Perform PCA.
    pca = PCA(n_components=6)
    mc = pca.fit_transform(mc)
    
    ## Take only the number of components explaining 90% of the variance.
    varexp = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(varexp >= 0.9) + 1
    mc = mc[:,:n_components]
    
    ## Save motion regressor.
    f = '%s/%s/msit_001/001/afMSIT.mc.par' %(mri_dir,subject)
    np.savetxt(f, mc, fmt='%s')
    info.write('%s,' %n_components)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Write scrubbers.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    for threshold in thresholds:
        
        ## Find threshold violations.
        if not threshold: ix, = np.where(fd >= np.inf)
        else: ix, = np.where(fd >= threshold)
                
        ## Save.
        info.write('%s,' %len(ix))
        f = '%s/%s/msit_001/001/afMSIT.censor.%s.par' %(mri_dir,subject,threshold)
        if len(ix): np.savetxt(f, tr_onsets[ix,np.newaxis], fmt='%s')

    info.write('\n')

info.close()
print 'Done.'

# Step 4: Compute Group Statistics

Following the formulas detailed in Pernet (2014): Misconceptions in the use of the General Linear Model applied to functional MRI: a tutorial for junior neuro-imagers

## Step 4a: Compute Beta Constants (Mean Signal)

In [ ]:
import os
import numpy as np
import nibabel as nib
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for subject in subjects:
    
    print subject,
    
    for hemi in ['lh','rh']:
        
        ## Load data.
        f = os.path.join(mri_dir, subject, 'msit_001', '001', 'fmcpr.sm6.fsaverage.%s.b0dc.nii.gz' %hemi)
        obj = nib.load(f)
        
        ## Extract data and average over acquisitions.
        data = obj.get_data()
        data = np.apply_over_axes(np.mean, data, -1)
        
        ## Save.
        f = os.path.join(mri_dir, subject, 'msit_001', 'afMSIT.6.0.5.%s' %hemi, 'betaconstant.nii.gz')
        obj = nib.Nifti1Image(data, obj.affine)
        nib.save(obj, f)
        
print 'Done.'

## Step 4b: Compute Percent Signal Change

In [ ]:
import os
import numpy as np
import nibabel as nib
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for subject in subjects:
    
    print subject,
    
    for hemi in ['lh','rh']:
        
        subj_dir = os.path.join(mri_dir, subject, 'msit_001', 'afMSIT.6.0.5.%s' %hemi)
        
        ## Load constants.
        design_matrix = np.loadtxt(os.path.join(subj_dir, 'X.dat'))
        mean_signal = nib.load(os.path.join(subj_dir, 'betaconstant.nii.gz')).get_data()
        
        ## Iteratively compute PSC across conditions.
        for n, con in enumerate(['Neu','Int']):
            
            ## Load contrast.
            ces = nib.load(os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'ces.nii.gz'))
            affine = ces.affine
            ces = ces.get_data()
            
            ## Compute PSC.
            sf = design_matrix.max(axis=0)[n]
            psc = ces * sf / mean_signal * 100.
            
            ## Save.
            psc = nib.Nifti1Image(psc, affine)
            nib.save(psc, os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'psc.nii.gz'))
            
print 'Done.'

## Step 4c: Compute Group Contrasts

In [ ]:
import os
import numpy as np
import nibabel as nib
from statsmodels.api import WLS
mri_dir = '/space/lilli/4/users/DARPA-MSIT'

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
subjects = np.loadtxt('scripts/fsfast/sessid.manual', dtype='str')

for hemi in ['lh','rh']:
    
    print 'Starting analysis of %s.' %hemi
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively load data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Loading subjects data.',
    
    psc, var = [], []
    for subject in subjects:
        
        ## Define subject directory.
        subj_dir = os.path.join(mri_dir, subject, 'msit_001', 'afMSIT.6.0.5.%s' %hemi)
        
        for con in ['Neu','Int']:
            
            ## Load percent signal change.
            f = os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'psc.nii.gz')
            psc.append( nib.load(f).get_data() )
            
            ## Load contrast variance.
            f = os.path.join(subj_dir, 'afMSIT.%s.par' %con, 'cesvar.nii.gz')
            obj = nib.load(f)
            var.append( obj.get_data() )
    
    ## Concatenate contrasts.
    psc = np.concatenate(psc, axis=-1).squeeze()
    var = np.concatenate(var, axis=-1).squeeze()
    affine = obj.affine
    print 'Finished.'
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Iteratively perform statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Compute contrast statistics.',
    
    ## Preallocate arrays.
    n_vert, n_cons = psc.shape
    Fvals, pvals = np.zeros((n_vert, 3)), np.zeros((n_vert, 3))

    ## Generate design matrix.
    X = np.zeros((n_cons,2))
    X[::2,0] = 1               # Neutral contrasts.
    X[1::2,1] = 1              # Interference contrasts.

    ## Generate weights.
    W = 1. / np.power(var, 2)

    ## Iterate over vertices.
    contrasts = [[1,0],[0,1],[-1,1]]
    for n in range(n_vert):

        ## Check if no data available.
        if np.any(np.isinf(W[n])): continue

        ## Fit model.
        model = WLS(psc[n], X, W[n]).fit()        
        
        ## Determine signs.
        signs = np.zeros(3)
        signs[:2] = np.sign(model.params)
        signs[-1] = np.sign(np.diff(model.params))
        
        ## Compute contrasts.
        for m, c in enumerate(contrasts):

            contrast = model.f_test(c)
            Fvals[n,m] = contrast.fvalue.max() * signs[m]
            pvals[n,m] = contrast.pvalue.max() * signs[m]

    ## Transform p-values.
    pvals = -np.log10(np.abs(pvals)) * np.sign(pvals)
    print 'Finished.'
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Save results.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    print 'Saving results.',
    
    for n, contrast in enumerate(['neu','int','diff']):
        
        ## Make out directory.
        out_dir = 'fmri/afMSIT.%s.%s' %(contrast,hemi)
        if not os.path.isdir(out_dir): os.makedirs(out_dir)
        
        ## Save F-statistics.
        F = Fvals[:,n]
        for _ in range(2): F = F[:,np.newaxis]
        obj = nib.Nifti1Image(F, affine)
        nib.save(obj, os.path.join(out_dir,'F.mgz'))
        
        ## Save p-values.
        p = pvals[:,n]
        for _ in range(2): p = p[:,np.newaxis]
        obj = nib.Nifti1Image(p, affine)
        nib.save(obj, os.path.join(out_dir,'sig.mgz'))
            
    print 'Finished.'
    
print 'Done.'